<a href="https://colab.research.google.com/github/ancestor9/Gyeongbok-AI-study/blob/main/2025%20AI%20Tutor/%EC%A0%9C1%EA%B0%95_LLM_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[All You Need to Know to Build Your First LLM App](https://medium.com/towards-data-science/all-you-need-to-know-to-build-your-first-llm-app-eb982c78ffac)**

<img src ='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*MKlUfYZdwSWpEulibj6S_g.png'>

### **1. Load documents using Langchain**

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/GPT-4"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

# find all the text on the page
text = soup.get_text()

# find the content div
content_div = soup.find('div', {'class': 'mw-parser-output'})

# remove unwanted elements from div
unwanted_tags = ['sup', 'span', 'table', 'ul', 'ol']
for tag in unwanted_tags:
    for match in content_div.findAll(tag):
        match.extract()

print(content_div.get_text())

2023 text-generating language model



Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model trained and created by OpenAI and the fourth in its series of GPT foundation models. It was launched on March 14, 2023, and made publicly available via the paid chatbot product ChatGPT Plus, via OpenAI's API, and via the free chatbot Microsoft Copilot.  As a transformer-based model, GPT-4 uses a paradigm where pre-training using both public data and "data licensed from third-party providers" is used to predict the next token. After this step, the model was then fine-tuned with reinforcement learning feedback from humans and AI for human alignment and policy compliance.
Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous iteration based on GPT-3.5, with the caveat that GPT-4 retains some of the problems with earlier revisions. GPT-4, equipped with vision capabilities (GPT-4V), is capable of taking images as input on ChatGPT.

<ipython-input-1-ca3c3c00c6fb>:18: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for match in content_div.findAll(tag):


### **2. Split our document into text fragments**
- **Next, we must divide the text into smaller sections called text chunks.**
- **Each text chunk represents a data point in the embedding space, allowing the computer to determine the similarity between these chunks.**

In [2]:
!pip install -q langchain

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


article_text = content_div.get_text()


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents([article_text])
print(len(texts))
print(texts[0])
print(texts[1])
print(texts[2])

260
page_content='2023 text-generating language model'
page_content='Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model trained and'
page_content='model trained and created by OpenAI and the fourth in its series of GPT foundation models. It was'


<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*YcleaA2sDs_IyjmrNzIajQ.png'>

### **3. From Text Chunks to Embeddings**
- openai 버젼은 1.0 이하 버젼으로해야 Embedding Model이 실행된다/


In [4]:
! pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [5]:
texts[0]

Document(metadata={}, page_content='2023 text-generating language model')

In [7]:
import openai
from google.colab import userdata
my_api_key = userdata.get('openapi')

# OpenAI API 키 설정
openai.api_key = my_api_key

print(texts[0])

embedding = openai.Embedding.create(
    input=texts[0].page_content, model="text-embedding-ada-002"
)["data"][0]["embedding"]

len(embedding)

page_content='2023 text-generating language model'


1536

In [8]:
embedding

[-0.032652031630277634,
 0.0001862208009697497,
 -0.0050887190736830235,
 0.020396990701556206,
 0.010535403154790401,
 0.03298893943428993,
 -0.03689146041870117,
 0.0010326590854674578,
 -0.026994777843356133,
 -0.020663708448410034,
 0.027696670964360237,
 0.020930428057909012,
 -0.0205514058470726,
 0.0032058930955827236,
 -0.003097099717706442,
 0.009286034852266312,
 0.0174630805850029,
 -0.006541636306792498,
 0.011518052779138088,
 0.0013818504521623254,
 0.01511876005679369,
 0.0025864732451736927,
 0.0004095102776773274,
 -0.007826099172234535,
 -0.0001937880733748898,
 0.009938795119524002,
 0.019681060686707497,
 -0.027865124866366386,
 0.0015292478492483497,
 -0.011012690141797066,
 0.018740523606538773,
 -0.006569711957126856,
 -0.012430512346327305,
 -0.006380200851708651,
 -0.01636812835931778,
 -0.01979336328804493,
 0.0004856217419728637,
 -0.021211184561252594,
 0.015582009218633175,
 -0.01356055960059166,
 0.021590206772089005,
 0.027837049216032028,
 0.017814027145

In [9]:
import numpy as np

np.array(embedding).shape

(1536,)

- We convert our text, such as the first text chunk containing “2023 text-generating language model,” into a vector with 1536 dimensions. By doing this for each text chunk, we can observe in a 1536-dimensional space which text chunks are closer and more similar to each other.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*lssUQDyZfz3MZCpCxIh-bw.png'>

#### **A commonly used distance metric is cosine similarity. So let’s try to calculate the cosine similarity between our question and the text chunks:**

In [10]:
import numpy as np
from numpy.linalg import norm
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openai

####################################################################
# load documents
####################################################################
# URL of the Wikipedia page to scrape
url = 'https://en.wikipedia.org/wiki/Prime_Minister_of_the_United_Kingdom'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the text on the page
text = soup.get_text()
print(len(text))
text[:1000]

104558


'\n\n\n\nPrime Minister of the United Kingdom - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\nToggle History subsection\n\n\n\n\n\n1.1\nOrigins: 1689–1742\n\n\n\n\n\n\n\n\n1.2\nDevelopme

In [11]:


####################################################################
# split text
####################################################################
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents([text[:1000]])

####################################################################
# calculate embeddings
####################################################################
# create new list with all text chunks
text_chunks=[]

for text in texts:
    text_chunks.append(text.page_content)

df = pd.DataFrame({'text_chunks': text_chunks})

####################################################################
# get embeddings from text-embedding-ada model
####################################################################
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df['ada_embedding'] = df.text_chunks.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

####################################################################
# calculate the embeddings for the user's question
####################################################################
users_question = "What is GPT-4?"

question_embedding = get_embedding(text=users_question, model="text-embedding-ada-002")

# create a list to store the calculated cosine similarity
cos_sim = []

for index, row in df.iterrows():
   A = row.ada_embedding
   B = question_embedding

   # calculate the cosine similarity
   cosine = np.dot(A,B)/(norm(A)*norm(B))

   cos_sim.append(cosine)

df["cos_sim"] = cos_sim
df.sort_values(by=["cos_sim"], ascending=False)

,text_chunks,ada_embedding,cos_sim
0,Prime Minister of the United Kingdom - Wikipedia,"[0.004429998341947794, -0.010896850377321243, ...",0.739559
6,Pages for logged out editors learn more\n\n\n\...,"[0.005005261395126581, 0.0037760180421173573, ...",0.730775
12,3\nConstitutional background,"[0.029426608234643936, 0.0021387115120887756, ...",0.724432
3,Contribute\n\t\n\n\nHelpLearn to editCommunity...,"[0.009443766437470913, 0.009263057261705399, 0...",0.719243
9,1.2\nDevelopment: 1742–1945\n\n\n\n\n\n\n\n\n\...,"[0.008957739919424057, -0.01072424091398716, 0...",0.716054
4,Search\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n...,"[0.007126160431653261, -0.0033666365779936314,...",0.715795
2,Navigation\n\t\n\n\nMain pageContentsCurrent e...,"[0.0027817003428936005, -0.0004660065169446170...",0.715466
7,Contents\nmove to sidebar\nhide\n\n\n\n\n(Top)...,"[0.016182171180844307, -0.0002322736254427582,...",0.713404
1,Jump to content\n\n\n\n\n\n\n\nMain menu\n\n\n...,"[0.014751596376299858, 0.006751743145287037, 0...",0.711497
11,2.3\nParliamentary powers\n\n\n\n\n\n\n\n\n2.4...,"[0.017178267240524292, -0.013773334212601185, ...",0.711047


### **4. Define the model you want to use**

In [12]:
!pip install -q langchain_community # Install the missing module

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [13]:
my_api_key[::3]

'spjl9vI4nOECQK6wWdWhNbRGITAyBkp7A3tPQ9GxGgQNqRmzBm8UAY8'

In [14]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key= my_api_key, temperature=0.7)

# 기본 모델 확인
print(llm.model_name)

gpt-3.5-turbo-instruct


<ipython-input-14-85fd30e5372c>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key= my_api_key, temperature=0.7)


In [15]:
models = openai.Model.list()
print([model['id'] for model in models['data']])

['gpt-4o-audio-preview-2024-12-17', 'dall-e-3', 'text-embedding-3-large', 'dall-e-2', 'o4-mini-2025-04-16', 'gpt-4o-audio-preview-2024-10-01', 'o4-mini', 'gpt-4.1-nano', 'gpt-4.1-nano-2025-04-14', 'gpt-4o-realtime-preview-2024-10-01', 'gpt-4o-realtime-preview', 'babbage-002', 'gpt-4', 'text-embedding-ada-002', 'gpt-4o-mini-audio-preview', 'gpt-4o-audio-preview', 'o1-preview-2024-09-12', 'o1-pro', 'o1-2024-12-17', 'gpt-4o-mini-realtime-preview', 'gpt-4.1-mini', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-3.5-turbo-instruct-0914', 'gpt-4o-mini-search-preview', 'o1', 'gpt-4.1-mini-2025-04-14', 'o1-pro-2025-03-19', 'chatgpt-4o-latest', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-4o-search-preview', 'gpt-4-turbo', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo', 'gpt-4-turbo-preview', 'gpt-4o-mini-search-preview-2025-03-11', 'gpt-4-0125-preview', 'gpt-4o-2024-11-20', 'whisper-1', 'gpt-4o-2024-05-13', 'gpt-4-turbo-2024-04-09', 'gpt-3.5-turbo-16k', 'o1-previe

### **5. Define our Prompt Template**

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# 특정 모델을 사용하도록 OpenAI LLM 초기화
llm = ChatOpenAI(openai_api_key=my_api_key,
                 model="gpt-4o-mini-2024-07-18",
                 temperature=0.7)

# 간단한 메시지로 채팅 모델 테스트
response = llm([HumanMessage(content="Hello, how are you?")])
print(response.content)

<ipython-input-16-358f1eb557a0>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=my_api_key,
<ipython-input-16-358f1eb557a0>:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content="Hello, how are you?")])


Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [17]:
users_question = "Who was the first Prime Minister of the UK?"

# 간단한 메시지로 채팅 모델 테스트
response = llm([HumanMessage(content=users_question)])
print(response.content)

The first Prime Minister of the United Kingdom is generally considered to be Sir Robert Walpole. He served from 1721 to 1742 and is often regarded as the de facto leader of the government during that time, although the title of "Prime Minister" was not officially used at that time.


### **6. Creating a vector store (vector database)**
#### **Hallucination!**

In [18]:
users_question = input("Enter your question: ")

# 간단한 메시지로 채팅 모델 테스트
response = llm([HumanMessage(content=users_question)])
print(response.content)

Enter your question: What is big data
Big data refers to the vast volumes of structured and unstructured data that are generated at high velocity and variety. It encompasses data sets that are so large or complex that traditional data processing software is inadequate to handle them. The concept of big data is often characterized by the "Three Vs":

1. **Volume**: This refers to the sheer amount of data generated from various sources, including social media, sensors, transactions, and more. Organizations are dealing with terabytes to petabytes of data.

2. **Velocity**: This pertains to the speed at which data is generated, processed, and analyzed. In many cases, data needs to be processed in real-time or near-real-time to derive actionable insights.

3. **Variety**: Big data comes in many formats, including structured data (like databases), semi-structured data (like XML or JSON), and unstructured data (like text, images, and videos). This variety makes it challenging to manage and an

### **6.1. Collect data that we want to use to answer the users’ questions:**

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ApbpqcZUMF-YaA6DbnVGww.png'>

In [19]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader


text = '''
조상구는 고려시대 명장으로 중국 수나라 100만 대군을 물리친 영웅이다.
'''

# Open a new file called 'output.txt' in write mode and store the file object in a variable
with open('output.txt', 'w', encoding='utf-8') as file:
    # Write the string to the file
    file.write(text)

### **6.2. Load the data and define how you want to split the data into text chunks**

### **PDF, Excel 등을 다양한 loader로 - Momgo DB의 Documents(Pages, metadata 로 구성)**

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*f5mcqjHkiz9QX63dpQnzxg.png'>

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load the document
with open('./output.txt', encoding='utf-8') as f:
    text = f.read()

# define the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.create_documents([text])
texts

[Document(metadata={}, page_content='조상구는 고려시대 명장으로 중국 수나라 100만 대군을 물리친 영웅이다.')]

### **6.3. Define the Embeddings Model you want to use to calculate the embeddings for your text chunks and store them in a vector store (here: Chroma)**

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ydbixXRwfgMYVdpctYTdew.png'>

- **최근에는 vector DB와 더불어 Graph DB를 적용  중**

In [21]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


In [22]:
!pip install -q tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.4 MB/s eta 0:00:00


In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# define the embeddings model
embeddings = OpenAIEmbeddings(openai_api_key=my_api_key)

# use the text chunks and the embeddings model to fill our vector store
db = Chroma.from_documents(texts, embeddings)

<ipython-input-23-9592e765b758>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=my_api_key)


In [24]:
db

### **6.4. Calculate the embeddings for the user’s question, find similar text chunks in our vector store and use them to build our prompt**

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*r2n4uA-ZlxZatnlhTVwv5Q.png'>

In [25]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
from langchain.schema import HumanMessage

user_question = "조상구에 대해 알려줘"

# use our vector store to find similar text chunks
results = db.similarity_search_with_score( # Use similarity_search_with_score directly
    query=user_question,
    k=5 # Pass the number of results using 'k'
)

# define the prompt template
template = """
You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."

Context sections:
{context}

Question:
{users_question}

Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])
# Create an LLMChain for easier prompt management
llm_chain = LLMChain(llm=llm, prompt=prompt)

# fill the prompt template
# Extract the document contents from the results
context_contents = [doc.page_content for doc, _ in results]
# Run the LLMChain with the formatted prompt
response = llm_chain.run(context=context_contents, users_question=user_question)

# Print the LLM's response
print(response)

<ipython-input-25-d02a7b78fcdc>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-25-d02a7b78fcdc>:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(context=context_contents, users_question=user_question)


조상구는 고려시대의 명장으로, 중국 수나라의 100만 대군을 물리친 영웅입니다.


## **Summary**

### - To enable our LLM to analyze and answer questions about our data, we usually don’t fine-tune the model. Instead, during **the fine-tuning process, the objective is to improve the model’s ability to effectively respond to a specific task, rather than teaching it new information.**

### - In the case of Alpaca 7B, the LLM (LLaMA) was fine-tuned to behave and interact like a chatbot. The focus was on refining the model’s responses, rather than teaching it completely new information.

### - So **to be able to answer questions about our own data, we use the Context Injection approach.** Creating an LLM app with Context Injection is a relatively simple process. **The main challenge lies in organizing and formatting the data to be stored in a vector database. This step is crucial for efficiently retrieving contextually similar information and ensuring reliable results.**

### - The goal of the article was **to demonstrate a minimalist approach to using embedding models, vector stores**, and LLMs to process user queries. It shows how these technologies can work together to provide relevant and accurate answers, even to constantly changing facts.